# Implementacja modelu uczenia maszynowego na potrzeby predykcji kursu indeksu WIG20

Ideą numer 1 jest próba wyłonienia najbardziej istotnych informacji, które zwiększają prawdopodobieństwo podjęcia właściwej decyzji inwestycyjnej.
Ideą numer 2 jest stworzenie modelu uczenia maszynowego, aby sprawdzić jego przydatność w tradingu. Używana do budowy modeli jest bibliteka scikit-learn.
Idea numer 3 to próba wyłonienia powiązań między wskaźnikami, które z wyprzedzeniem informują o zbliżających się sygnałach inwestycyjnych

### Instalacja niezbędnych bibliotek

Bazuję na danych historycznych pochodzących z serwisu: https://www.dukascopy.com/swiss/pl/marketwatch/historical/
Dukascopy daje możliwość pobierania danych za pomocą specjalnej biblioteki pythonowej, co jest bardzo wygodnym rozwiązaniem. Nazwa biblioteki: dukascopy-python

In [34]:
#pip install dukascopy-python

Biblioteka dukascopy-python pobiera dane prosto do struktury danych Pandas DataFrame, więc trzeba zainstalować biblitekę Pandas

In [35]:
#pip install pandas

Sprawdzenie możliwości biblioteki

Niezbędna będzie malipulacja datami: biblioteka datetime

In [36]:
#pip install datetime

Skoro ML to biblioteka scikit-learn

In [37]:
#pip install scikit-learn

Jakaś biblioteka do analizy technicznej

In [38]:
#pip install pandas-ta

### Import bibliotek

In [39]:
from datetime import datetime
#
import dukascopy_python as dcp
#
import pandas as pd
#
import pandas_ta as ta

Wyświetlenie dokumentacji biblioteki dukascopy_python

In [40]:
help(dcp)

Help on package dukascopy_python:

NAME
    dukascopy_python

PACKAGE CONTENTS
    instruments

FUNCTIONS
    fetch(instrument: str, interval: str, offer_side: str, start: datetime.datetime, end: datetime.datetime, max_retries: int = 7, limit: int = 30000, debug=False)

    live_fetch(instrument: str, interval_value: int, time_unit: str, offer_side: str, start: datetime.datetime, end: datetime.datetime, max_retries: int = 7, limit: int = 30000, debug=False)

    sleep(...)
        sleep(seconds)

        Delay execution for a given number of seconds.  The argument may be
        a floating-point number for subsecond precision.

DATA
    INTERVAL_DAY_1 = '1DAY'
    INTERVAL_HOUR_1 = '1HOUR'
    INTERVAL_HOUR_4 = '4HOUR'
    INTERVAL_MIN_1 = '1MIN'
    INTERVAL_MIN_10 = '10MIN'
    INTERVAL_MIN_15 = '15MIN'
    INTERVAL_MIN_30 = '30MIN'
    INTERVAL_MIN_5 = '5MIN'
    INTERVAL_MONTH_1 = '1MONTH'
    INTERVAL_SEC_1 = '1SEC'
    INTERVAL_SEC_10 = '10SEC'
    INTERVAL_SEC_30 = '30SEC'
    I

Sprawdzenie listy dostępnych instrumentów

In [41]:
import dukascopy_python.instruments as dcpi

In [42]:
help(dcpi)

Help on module dukascopy_python.instruments in dukascopy_python:

NAME
    dukascopy_python.instruments

DATA
    INSTRUMENT_AUSTRIA_ANDR_AT_EUR = 'ANDR.AT/EUR'
    INSTRUMENT_AUSTRIA_EBS_AT_EUR = 'EBS.AT/EUR'
    INSTRUMENT_AUSTRIA_POST_AT_EUR = 'POST.AT/EUR'
    INSTRUMENT_AUSTRIA_RBI_AT_EUR = 'RBI.AT/EUR'
    INSTRUMENT_AUSTRIA_TKA_AT_EUR = 'TKA.AT/EUR'
    INSTRUMENT_AUSTRIA_VER_AT_EUR = 'VER.AT/EUR'
    INSTRUMENT_AUSTRIA_VIG_AT_EUR = 'VIG.AT/EUR'
    INSTRUMENT_AUSTRIA_VOE_AT_EUR = 'VOE.AT/EUR'
    INSTRUMENT_AUSTRIA_WIE_AT_EUR = 'WIE.AT/EUR'
    INSTRUMENT_BELGIUM_ABI_BE_EUR = 'ABI.BE/EUR'
    INSTRUMENT_BELGIUM_AGS_BE_EUR = 'AGS.BE/EUR'
    INSTRUMENT_BELGIUM_BELG_BE_EUR = 'BELG.BE/EUR'
    INSTRUMENT_BELGIUM_KBC_BE_EUR = 'KBC.BE/EUR'
    INSTRUMENT_BELGIUM_SOLB_BE_EUR = 'SOLB.BE/EUR'
    INSTRUMENT_BELGIUM_UCB_BE_EUR = 'UCB.BE/EUR'
    INSTRUMENT_BELGIUM_UMI_BE_EUR = 'UMI.BE/EUR'
    INSTRUMENT_BND_CFD_BUND_TR_EUR = 'BUND.TR/EUR'
    INSTRUMENT_BND_CFD_UKGILT_TR_GBP = 'UKGILT.

Sprawdzenie możliwości biblioteki Pandas TA

In [43]:
help(ta)

Help on package pandas_ta:

NAME
    pandas_ta

PACKAGE CONTENTS
    candles (package)
    core
    custom
    cycles (package)
    momentum (package)
    overlap (package)
    performance (package)
    statistics (package)
    trend (package)
    utils (package)
    volatility (package)
    volume (package)

SUBMODULES
    alphavantage
    data
    yahoofinance

FUNCTIONS
    localtime(...)
        localtime([seconds]) -> (tm_year,tm_mon,tm_mday,tm_hour,tm_min,
                                  tm_sec,tm_wday,tm_yday,tm_isdst)

        Convert seconds since the Epoch to a time tuple expressing local time.
        When 'seconds' is not passed in, convert the current time instead.

    mfloor = floor(x, /)
        Return the floor of x as an Integral.

        This is the largest integer <= x.

    mul(a, b, /)
        Same as a * b.

    npArray = array(...)
        array(object, dtype=None, *, copy=True, order='K', subok=False, ndmin=0,
              like=None)

        Create an arra

Sprawdzenie jakie wskaźniki i funkcje są dostępne w bibliotece Pandas-TA

In [50]:
lista_wskaźników = [metoda for metoda in dir(df.ta) if not metoda.startswith('_')]

print(f"Znaleziono {len(lista_wskaźników)} wskaźników i funkcji.")
print(lista_wskaźników)

Znaleziono 157 wskaźników i funkcji.
['aberration', 'above', 'above_value', 'accbands', 'ad', 'adjusted', 'adosc', 'adx', 'alma', 'amat', 'ao', 'aobv', 'apo', 'aroon', 'atr', 'bbands', 'below', 'below_value', 'bias', 'bop', 'brar', 'categories', 'cci', 'cdl_pattern', 'cdl_z', 'cfo', 'cg', 'chop', 'cksp', 'cmf', 'cmo', 'constants', 'coppock', 'cores', 'cross', 'cross_value', 'cti', 'datetime_ordered', 'decay', 'decreasing', 'dema', 'dm', 'donchian', 'dpo', 'ebsw', 'efi', 'ema', 'entropy', 'eom', 'er', 'eri', 'exchange', 'fisher', 'fwma', 'ha', 'hilo', 'hl2', 'hlc3', 'hma', 'hwc', 'hwma', 'ichimoku', 'increasing', 'indicators', 'inertia', 'jma', 'kama', 'kc', 'kdj', 'kst', 'kurtosis', 'kvo', 'last_run', 'linreg', 'log_return', 'long_run', 'macd', 'mad', 'massi', 'mcgd', 'median', 'mfi', 'midpoint', 'midprice', 'mom', 'natr', 'nvi', 'obv', 'ohlc4', 'pdist', 'percent_return', 'pgo', 'ppo', 'psar', 'psl', 'pvi', 'pvo', 'pvol', 'pvr', 'pvt', 'pwma', 'qqe', 'qstick', 'quantile', 'reverse', 'r

### Definicja paramertów dla dla funkcji fetch() z biblioteki dukascopy_python

In [45]:
instrument = 'EUR/USD'
interval = '1DAY'
offer_side = 'B' 
start_date = datetime(2000, 1, 1)
end_date = datetime.today()

### Pobranie danych do Pandas DataFrame

In [46]:
df = dcp.fetch(instrument, interval, offer_side, start_date, end_date)

INFO:DUKASCRIPT:current timestamp :2025-07-29T02:00:00


In [47]:
df.head()

,open,high,low,close,volume
timestamp,,,,,
2003-05-04 00:00:00+00:00,1.12273,1.12338,1.12160,1.12169,257040.5
2003-05-05 00:00:00+00:00,1.12161,1.13009,1.12014,1.12924,1276005.6
2003-05-06 00:00:00+00:00,1.12921,1.14506,1.12723,1.14234,1281467.0
2003-05-07 00:00:00+00:00,1.14218,1.14323,1.13265,1.13494,1272596.6
2003-05-08 00:00:00+00:00,1.13507,1.15077,1.13006,1.14820,1266581.3


In [54]:
df.tail()

,open,high,low,close,volume
timestamp,,,,,
2025-07-24 00:00:00+00:00,1.17732,1.17887,1.17303,1.17545,89743.31
2025-07-25 00:00:00+00:00,1.17548,1.17609,1.17028,1.17396,84426.67
2025-07-27 00:00:00+00:00,1.17552,1.17686,1.17507,1.17554,4400.20
2025-07-28 00:00:00+00:00,1.17554,1.17700,1.15850,1.15963,94411.61
2025-07-29 00:00:00+00:00,1.15960,1.15989,1.15188,1.15506,106375.16


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7149 entries, 2003-05-04 00:00:00+00:00 to 2025-07-29 00:00:00+00:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    7149 non-null   float64
 1   high    7149 non-null   float64
 2   low     7149 non-null   float64
 3   close   7149 non-null   float64
 4   volume  7149 non-null   float64
dtypes: float64(5)
memory usage: 335.1 KB


Zbiór jest kompletny, zgodnie z oczekiwaniami zawiera dane do dnia poprzedzającego dzień uruchomienia skryptu.
Pora zatem na rozbudowanie zbioru i policzenie interesujących nas statystyk.

In [74]:
help(ta.cdl_pattern)

Help on function cdl_pattern in module pandas_ta.candles.cdl_pattern:

cdl_pattern(open_, high, low, close, name: Union[str, Sequence[str]] = 'all', scalar=None, offset=None, **kwargs) -> pandas.core.frame.DataFrame
    Candle Pattern

    A wrapper around all candle patterns.

    Examples:

    Get all candle patterns (This is the default behaviour)
    >>> df = df.ta.cdl_pattern(name="all")
    Or
    >>> df.ta.cdl("all", append=True) # = df.ta.cdl_pattern("all", append=True)

    Get only one pattern
    >>> df = df.ta.cdl_pattern(name="doji")
    Or
    >>> df.ta.cdl("doji", append=True)

    Get some patterns
    >>> df = df.ta.cdl_pattern(name=["doji", "inside"])
    Or
    >>> df.ta.cdl(["doji", "inside"], append=True)

    Args:
        open_ (pd.Series): Series of 'open's
        high (pd.Series): Series of 'high's
        low (pd.Series): Series of 'low's
        close (pd.Series): Series of 'close's
        name: (Union[str, Sequence[str]]): name of the patterns
        sca

In [75]:
help(ta.cdl_z)

Help on function cdl_z in module pandas_ta.candles.cdl_z:

cdl_z(open_, high, low, close, length=None, full=None, ddof=None, offset=None, **kwargs)
    Candle Type: Z

    Normalizes OHLC Candles with a rolling Z Score.

    Source: Kevin Johnson

    Calculation:
        Default values:
            length=30, full=False, ddof=1
        Z = ZSCORE

        open  = Z( open, length, ddof)
        high  = Z( high, length, ddof)
        low   = Z(  low, length, ddof)
        close = Z(close, length, ddof)

    Args:
        open_ (pd.Series): Series of 'open's
        high (pd.Series): Series of 'high's
        low (pd.Series): Series of 'low's
        close (pd.Series): Series of 'close's
        length (int): The period. Default: 10

    Kwargs:
        naive (bool, optional): If True, prefills potential Doji less than
            the length if less than a percentage of it's high-low range.
            Default: False
        fillna (value, optional): pd.DataFrame.fillna(value)
        fi

In [76]:
help(ta.cfo)

Help on function cfo in module pandas_ta.momentum.cfo:

cfo(close, length=None, scalar=None, drift=None, offset=None, **kwargs)
    Chande Forcast Oscillator (CFO)

    The Forecast Oscillator calculates the percentage difference between the actual
    price and the Time Series Forecast (the endpoint of a linear regression line).

    Sources:
        https://www.fmlabs.com/reference/default.htm?url=ForecastOscillator.htm

    Calculation:
        Default Inputs:
            length=9, drift=1, scalar=100
        LINREG = Linear Regression

        CFO = scalar * (close - LINERREG(length, tdf=True)) / close

    Args:
        close (pd.Series): Series of 'close's
        length (int): The period. Default: 9
        scalar (float): How much to magnify. Default: 100
        drift (int): The short period. Default: 1
        offset (int): How many periods to offset the result. Default: 0

    Kwargs:
        fillna (value, optional): pd.DataFrame.fillna(value)
        fill_method (value, op

In [77]:
help(ta.cg)

Help on function cg in module pandas_ta.momentum.cg:

cg(close, length=None, offset=None, **kwargs)
    Center of Gravity (CG)

    The Center of Gravity Indicator by John Ehlers attempts to identify turning
    points while exhibiting zero lag and smoothing.

    Sources:
        http://www.mesasoftware.com/papers/TheCGOscillator.pdf

    Calculation:
        Default Inputs:
            length=10

    Args:
        close (pd.Series): Series of 'close's
        length (int): The length of the period. Default: 10
        offset (int): How many periods to offset the result. Default: 0

    Kwargs:
        fillna (value, optional): pd.DataFrame.fillna(value)
        fill_method (value, optional): Type of fill method

    Returns:
        pd.Series: New feature generated.



In [78]:
help(ta.chop)

Help on function chop in module pandas_ta.trend.chop:

chop(high, low, close, length=None, atr_length=None, ln=None, scalar=None, drift=None, offset=None, **kwargs)
    Choppiness Index (CHOP)

    The Choppiness Index was created by Australian commodity trader
    E.W. Dreiss and is designed to determine if the market is choppy
    (trading sideways) or not choppy (trading within a trend in either
    direction). Values closer to 100 implies the underlying is choppier
    whereas values closer to 0 implies the underlying is trending.

    Sources:
        https://www.tradingview.com/scripts/choppinessindex/
        https://www.motivewave.com/studies/choppiness_index.htm

    Calculation:
        Default Inputs:
            length=14, scalar=100, drift=1
        HH = high.rolling(length).max()
        LL = low.rolling(length).min()

        ATR_SUM = SUM(ATR(drift), length)
        CHOP = scalar * (LOG10(ATR_SUM) - LOG10(HH - LL))
        CHOP /= LOG10(length)

    Args:
        high (

In [79]:
help(ta.cksp)

Help on function cksp in module pandas_ta.trend.cksp:

cksp(high, low, close, p=None, x=None, q=None, tvmode=None, offset=None, **kwargs)
    Chande Kroll Stop (CKSP)

    The Tushar Chande and Stanley Kroll in their book
    “The New Technical Trader”. It is a trend-following indicator,
    identifying your stop by calculating the average true range of
    the recent market volatility. The indicator defaults to the implementation
    found on tradingview but it provides the original book implementation as well,
    which differs by the default periods and moving average mode. While the trading
    view implementation uses the Welles Wilder moving average, the book uses a
    simple moving average.

    Sources:
        https://www.multicharts.com/discussion/viewtopic.php?t=48914
        "The New Technical Trader", Wikey 1st ed. ISBN 9780471597803, page 95

    Calculation:
        Default Inputs:
            p=10, x=1, q=9, tvmode=True
        ATR = Average True Range

        LS0 = h

In [80]:
help(ta.cmf)

Help on function cmf in module pandas_ta.volume.cmf:

cmf(high, low, close, volume, open_=None, length=None, offset=None, **kwargs)
    Chaikin Money Flow (CMF)

    Chailin Money Flow measures the amount of money flow volume over a specific
    period in conjunction with Accumulation/Distribution.

    Sources:
        https://www.tradingview.com/wiki/Chaikin_Money_Flow_(CMF)
        https://stockcharts.com/school/doku.php?id=chart_school:technical_indicators:chaikin_money_flow_cmf

    Calculation:
        Default Inputs:
            length=20
        if 'open':
            ad = close - open
        else:
            ad = 2 * close - high - low

        hl_range = high - low
        ad = ad * volume / hl_range
        CMF = SUM(ad, length) / SUM(volume, length)

    Args:
        high (pd.Series): Series of 'high's
        low (pd.Series): Series of 'low's
        close (pd.Series): Series of 'close's
        volume (pd.Series): Series of 'volume's
        open_ (pd.Series): Series o

In [81]:
help(ta.cmo)

Help on function cmo in module pandas_ta.momentum.cmo:

cmo(close, length=None, scalar=None, talib=None, drift=None, offset=None, **kwargs)
    Chande Momentum Oscillator (CMO)

    Attempts to capture the momentum of an asset with overbought at 50 and
    oversold at -50.

    Sources:
        https://www.tradingtechnologies.com/help/x-study/technical-indicator-definitions/chande-momentum-oscillator-cmo/
        https://www.tradingview.com/script/hdrf0fXV-Variable-Index-Dynamic-Average-VIDYA/

    Calculation:
        Default Inputs:
            drift=1, scalar=100

        # Same Calculation as RSI except for this step
        CMO = scalar * (PSUM - NSUM) / (PSUM + NSUM)

    Args:
        close (pd.Series): Series of 'close's
        scalar (float): How much to magnify. Default: 100
        talib (bool): If TA Lib is installed and talib is True, Returns the TA Lib
            version. If TA Lib is not installed but talib is True, it runs the Python
            version TA Lib. Defaul

In [82]:
help(ta.constants)

AttributeError: module 'pandas_ta' has no attribute 'constants'

In [ ]:
help(ta.coppock)

In [ ]:
help(ta.cores)

In [ ]:
help(ta.cross)

In [ ]:
help(ta.cross_value)

In [ ]:
help(ta.cti)

In [ ]:
help(ta.datetime_ordered)

In [ ]:
help(ta.decay)

In [ ]:
help(ta.decreasing)

In [ ]:
help(ta.dema)

In [ ]:
help(ta.dm)

In [ ]:
help(ta.donchian)

In [ ]:
help(ta.dpo)

In [ ]:
help(ta.ebsw)

In [ ]:
help(ta.efi)

In [ ]:
help(ta.ema)

In [ ]:
help(ta.entropy)

In [ ]:
help(ta.eom)

In [ ]:
help(ta.er)

In [ ]:
help(ta.eri)

In [ ]:
help(ta.exchange)

In [ ]:
help(ta.fisher)

In [ ]:
help(ta.fwma)

In [ ]:
help(ta.ha)

In [ ]:
help(ta.hilo)

In [ ]:
help(ta.hl2)

In [ ]:
help(ta.hlc3)

In [ ]:
help(ta.hma)

In [ ]:
help(ta.hwc)

In [ ]:
help(ta.hwma)

In [ ]:
help(ta.ichimoku)

In [ ]:
help(ta.increasing)

In [ ]:
help(ta.indicators)

In [ ]:
help(ta.inertia)

In [ ]:
help(ta.jma)

In [ ]:
help(ta.kama)

In [ ]:
help(ta.kc)

In [ ]:
help(ta.kdj)

In [ ]:
help(ta.kst)

In [ ]:
help(ta.kurtosis)

In [ ]:
help(ta.kvo)

In [ ]:
help(ta.last_run)

In [ ]:
help(ta.linreg)

In [ ]:
help(ta.log_return)

In [ ]:
help(ta.long_run)

In [ ]:
help(ta.macd)

In [ ]:
help(ta.mad)

In [ ]:
help(ta.massi)

In [ ]:
help(ta.mcgd)

In [ ]:
help(ta.median)

In [ ]:
help(ta.mfi)

In [ ]:
help(ta.midpoint)

In [ ]:
help(ta.midprice)

In [ ]:
help(ta.mom)

In [ ]:
help(ta.natr)

In [ ]:
help(ta.nvi)

In [ ]:
help(ta.obv)

In [ ]:
help(ta.ohlc4)

In [ ]:
help(ta.pdist)

In [ ]:
help(ta.percent_return)

In [ ]:
help(ta.pgo)

In [ ]:
help(ta.ppo)

In [ ]:
help(ta.psar)

In [ ]:
help(ta.psl)

In [ ]:
help(ta.pvi)

In [ ]:
help(ta.pvo)

In [ ]:
help(ta.pvol)

In [ ]:
help(ta.pvr)

In [ ]:
help(ta.pvt)

In [ ]:
help(ta.pwma)

In [ ]:
help(ta.qqe)

In [ ]:
help(ta.qstick)

In [ ]:
help(ta.quantile)

In [ ]:
help(ta.reverse)

In [ ]:
help(ta.rma)

In [ ]:
help(ta.roc)

In [ ]:
help(ta.rsi)

In [ ]:
help(ta.rsx)

In [ ]:
help(ta.rvgi)

In [ ]:
help(ta.rvi)

In [ ]:
help(ta.short_run)

In [ ]:
help(ta.sinwma)

In [ ]:
help(ta.skew)

In [ ]:
help(ta.slope)

In [ ]:
help(ta.sma)

In [ ]:
help(ta.smi)

In [ ]:
help(ta.squeeze)

In [ ]:
help(ta.squeeze_pro)

In [ ]:
help(ta.ssf)

In [ ]:
help(ta.stc)

In [ ]:
help(ta.stdev)

In [ ]:
help(ta.stoch)

In [ ]:
help(ta.stochrsi)

In [ ]:
help(ta.strategy)

In [ ]:
help(ta.supertrend)

In [ ]:
help(ta.swma)

In [ ]:
help(ta.t3)

In [ ]:
help(ta.td_seq)

In [ ]:
help(ta.tema)

In [ ]:
help(ta.thermo)

In [ ]:
help(ta.ticker)

In [ ]:
help(ta.time_range)

In [ ]:
help(ta.to_utc)

In [ ]:
help(ta.tos_stdevall)

In [ ]:
help(ta.trima)

In [ ]:
help(ta.trix)

In [ ]:
help(ta.true_range)

In [ ]:
help(ta.tsi)

In [ ]:
help(ta.tsignals)

In [ ]:
help(ta.ttm_trend)

In [ ]:
help(ta.ui)

In [ ]:
help(ta.uo)

In [ ]:
help(ta.variance)

In [ ]:
help(ta.version)

In [ ]:
help(ta.vhf)

In [ ]:
help(ta.vidya)

In [ ]:
help(ta.vortex)

In [ ]:
help(ta.vp)

In [ ]:
help(ta.vwap)

In [ ]:
help(ta.vwma)

In [ ]:
help(ta.wcp)

In [ ]:
help(ta.willr)

In [ ]:
help(ta.wma)

In [ ]:
help(ta.xsignals)

In [ ]:
help(ta.zlma)

In [ ]:
help(ta.zscore)

In [ ]:
aberration
accbands
ad
adosc
adx
alma
amat
ao
aobv
apo
aroon
atr
bbands
bias
bop
brar
cci